## Init


In [ ]:
import requests
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
import os
from dotenv import load_dotenv

load_dotenv()

cred = credentials.Certificate("app/orbit-d8ec9-firebase-adminsdk-3mpgc-334631a536.json")
firebase_admin.initialize_app(cred)

## Celstark

In [ ]:
def fetch_celestrak_data():
    url = "https://www.celestrak.com/NORAD/elements/active.txt"
    response = requests.get(url)
    data = response.text
    with open("data/celestrak_data.txt", "w") as file:
        file.write(data)
    return data

## Spacetrack


In [ ]:
def fetch_space_track_data():
    username = os.getenv("SPACE_TRACK_USERNAME")
    password = os.getenv("SPACE_TRACK_PASSWORD")

    session = requests.Session()
    session.auth = (username, password)

    login_url = "https://www.space-track.org/auth/login"
    response = session.post(login_url)

    if response.status_code != 200:
        print("Login failed")
        return None

    data_url = "https://www.space-track.org/your-data-endpoint"
    data_response = session.get(data_url)

    if data_response.status_code != 200:
        print("Failed to fetch data from Space-Track")
        return None

    data = data_response.text
    with open("data/space_track_data.txt", "w") as file:
        file.write(data)
    return data

## Save

In [ ]:

def save_to_firebase():
    db = firestore.client()
    celestrak_data = pd.read_csv("data/celestrak_data.txt", sep="\s+", header=None)
    celestrak_ref = db.collection("celestrak_data")
    for index, row in celestrak_data.iterrows():
        celestrak_ref.add(row.to_dict())

    space_track_data = pd.read_csv("data/space_track_data.txt", sep="\s+", header=None)
    space_track_ref = db.collection("space_track_data")
    for index, row in space_track_data.iterrows():
        space_track_ref.add(row.to_dict())